In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# Liczba symulacji
num_simulations = 1000

# Liczba próbek
sample_sizes = np.arange(10, 1000, 50)

# Parametry dla rozkładów
parameters = [(2, 1), (3, 2), (4, 3), (5, 4), (6, 5)]  # Zmienione parametry

# Inicjalizacja wyników
power_kolmogorov_nig = np.zeros((len(parameters), len(sample_sizes)))
power_kolmogorov_pit_nig = np.zeros((len(parameters), len(sample_sizes)))
power_kolmogorov_gamma = np.zeros((len(parameters), len(sample_sizes)))
power_kolmogorov_pit_gamma = np.zeros((len(parameters), len(sample_sizes)))

# Dla każdego zestawu parametrów
for i, param in enumerate(parameters):
    # Dla każdej wielkości próbki
    for j, n in enumerate(sample_sizes):
        # Generuj dane z rozkładu NIG
        data_nig = stats.norminvgauss.rvs(a=param[0], b=param[1], size=(num_simulations, n))

        # Generuj dane z rozkładu Gamma
        data_gamma = stats.gamma.rvs(a=param[0], scale=param[1], size=(num_simulations, n))

        # Test Kołmogorowa dla NIG
        p_kolmogorov_nig = np.array([stats.kstest(data, 'norm')[1] for data in data_nig])
        count_kolmogorov_nig = np.sum(p_kolmogorov_nig < 0.05)

        # Test Kołmogorowa dla Gamma
        p_kolmogorov_gamma = np.array([stats.kstest(data, 'norm')[1] for data in data_gamma])
        count_kolmogorov_gamma = np.sum(p_kolmogorov_gamma < 0.05)

        # Test Kołmogorowa z PIT dla NIG
        data_pit_nig = stats.norm.ppf(stats.norminvgauss.cdf(data_nig, a=param[0], b=param[1]))
        p_kolmogorov_pit_nig = np.array([stats.kstest(data, 'norm')[1] for data in data_pit_nig])
        count_kolmogorov_pit_nig = np.sum(p_kolmogorov_pit_nig < 0.05)

        # Test Kołmogorowa z PIT dla Gamma
        data_pit_gamma = stats.norm.ppf(stats.gamma.cdf(data_gamma, a=param[0], scale=param[1]))
        p_kolmogorov_pit_gamma = np.array([stats.kstest(data, 'norm')[1] for data in data_pit_gamma])
        count_kolmogorov_pit_gamma = np.sum(p_kolmogorov_pit_gamma < 0.05)

        # Oblicz moc testów
        power_kolmogorov_nig[i, j] = count_kolmogorov_nig / num_simulations
        power_kolmogorov_pit_nig[i, j] = count_kolmogorov_pit_nig / num_simulations
        power_kolmogorov_gamma[i, j] = count_kolmogorov_gamma / num_simulations
        power_kolmogorov_pit_gamma[i, j] = count_kolmogorov_pit_gamma / num_simulations

# Rysowanie wykresów
plt.figure(figsize=(15, 10))  # Zwiększ rozmiar wykresu
for i, param in enumerate(parameters):
    plt.plot(sample_sizes, power_kolmogorov_nig[i, :], label=f'NIG, param={param}, Kolmogorov')
    plt.plot(sample_sizes, power_kolmogorov_pit_nig[i, :], label=f'NIG, param={param}, Kolmogorov-PIT')
    plt.plot(sample_sizes, power_kolmogorov_gamma[i, :], label=f'Gamma, param={param}, Kolmogorov')
    plt.plot(sample_sizes, power_kolmogorov_pit_gamma[i, :], label=f'Gamma, param={param}, Kolmogorov-PIT')
plt.xlabel('Liczba danych')
plt.ylabel('Moc testu')
plt.title('Moc testu Kołmogorowa i testu Kołmogorowa z PIT dla danych z rozkładów NIG i Gamma')
plt.legend(bbox_to_anchor=(1.1, 1), loc='upper left')  # Przesuń legendę nieco dalej
plt.grid(True)
plt.show()
